In [372]:
import pandas as pd
import numpy as np
import re
import dask.dataframe as dd
from dask.multiprocessing import get
from tqdm import tqdm

In [373]:
def syn_gen(col_name):
    lst = [None]*25
    if col_name == 'nan':
        return ['nan']
    col_name = str(col_name)
    col_name = col_name.lower()
    name = col_name.split(',')[0]
    names = []
    names.append(name)                                           # As given
    
    name = re.subn('of|\sand|&', '', name, flags = re.IGNORECASE)[0]
    
    name = re.subn('\s+', ' ', name)[0]                          # Removing 'of' and 'and'
    names.append(name)
    names.append(''.join([word[0] for word in name.split()]).upper())
    
    if "'" in col_name:    
        names.append(''.join([word[0] for word in filter(None,re.split('\W+', name))]).upper())
    if col_name.count('.') > 1:
        names.append(''.join(re.split(r'\.\s?', col_name)[:-1]).upper())
        
        try:
            pattern = re.compile('\s?univ\S*\s?|\s?coll\S*\s?|\s?inst\S*\s?|\sres\S*\sfound\S*\s?')
            inst_type = pattern.findall(name)[0]
            names.append(names[-1]+inst_type)
        except:
            pass
        
    if ',' in col_name:
        location_full = [name+re.findall(',.*', col_name)[0][1:] for name in names]
        try:
            location_short = [name+col_name.split(',')[-1].split()[0][0].upper() for name in filter(lambda x: x.isupper(), names)]
            location_short.extend([name+'-'+col_name.split(',')[-1].split()[0][0].upper() for name in filter(lambda x: x.isupper(), names)])
            location_short.extend([name+'/'+col_name.split(',')[-1].split()[0][0].upper() for name in filter(lambda x: x.isupper(), names)])
            location_short.extend([name+' '+col_name.split(',')[-1].split()[0][0].upper() for name in filter(lambda x: x.isupper(), names)])
            names.extend(location_short)
        except:
            pass
        names.extend(location_full)
    return (list(set(names))+lst)[:25]

In [374]:
def rm_brk(s):
    s = str(s)
    return re.sub('\\(.+?\\)','',s)

def ret_zero(lst):
    return lst[0]

In [375]:
data = pd.read_csv('../data/all_institutions_live.csv', engine='python')
data['name_clean'] = data.name

In [376]:
data.name_clean = data.name.apply(rm_brk)
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip(',')
data.name_clean = data.name_clean.str.rstrip('.')
data.name_clean = data.name_clean.str.rstrip('-')

In [377]:
data['name_clean'] = data.name_clean.str.split('Ph:|PH[0-9]|Ph[0-9]|Ph. No|Ph\.|Mob\.No\.|PH\.|ph\.|Address|Dist.|Ward No|Distt\.|Distt|H\.No|PIN-|PIN [0-9]|PIN -|Pin-|village|VILLAGE|Write institute Name here|near|Near|NEAR|DIST-|City Off|DIST:|DIST\.|D\.No').apply(ret_zero)

data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip(',')
data.name_clean = data.name_clean.str.rstrip('.')
data.name_clean = data.name_clean.str.rstrip('-')

In [378]:
def rep_sub_none(string):
    string = str(string)
    return re.sub('[0-9]{3} [0-9]{3}-|[0-9]{6}-|Dist','',string)

def rep_sub_quote(string):
    string = str(string)
    string = re.sub(',\.|PINCODE|NH-[0-9]{1,4},|NH-[0-9]{1,4}|NH[0-9]{1,4}|^[A-Z][0-9]{1,4} |^[A-Z][0-9]{1,4}','',string)
    return re.sub(';|:','\'',string)

def rep_sub_fs(string):
    string = str(string)
    return re.sub('\.\.|>','.',string)

def rep_sub_ch(string):
    string = str(string)
    string =  re.sub('---|--','-',string)
    return re.sub(',,,|,,|<',',',string)

def rep_digit(string):
    string = str(string)
    return re.sub('[0-9]{1,4}$|[0-9]{1,4}.$','',string)

In [379]:
data.name_clean = data.name_clean.apply(rep_sub_none)

data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip(',')
data.name_clean = data.name_clean.str.rstrip('.')
data.name_clean = data.name_clean.str.rstrip('-')

In [380]:
data['name_clean'] = data.name_clean.str.split('-[0-9]{3} [0-9]{3}|-[0-9]{6}|P\.O\.|P\.O|PB\.NO|- [0-9]{3} [0-9]{3}|- [0-9]{6}|[0-9]{3} [0-9]{3}|[0-9]{6}|Ta:|Di:|TAL:|OPP:|Vill|near|Post Box|P\.B\. No|Tq:|Dt:|Tq.:|Opp:|Opp\.|opp\.|Tal:|Di stt:|TQ:|At:|CITY:|Plot|PLOT|Opposite|Opp\.|OPP\.').apply(ret_zero)
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip(',')
data.name_clean = data.name_clean.str.rstrip('.')
data.name_clean = data.name_clean.str.rstrip('-')

data['name_clean'] = data.name_clean.str.split('P\.B\.No\.|P B No\.|No\.|plot|Tal\.|DISTT|VILL-|VILL\.|NO\.|VILL |POST\.|TQ\.|OPP |OPP-|TAL-|TAL\.|AT\.|#|S NO').apply(ret_zero)
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip(',')
data.name_clean = data.name_clean.str.rstrip('.')
data.name_clean = data.name_clean.str.rstrip('-')

In [381]:
data.name_clean = data.name_clean.apply(rep_sub_quote)

data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip(',')
data.name_clean = data.name_clean.str.rstrip('.')
data.name_clean = data.name_clean.str.rstrip('-')

data.name_clean = data.name_clean.apply(rep_sub_fs)

data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip(',')
data.name_clean = data.name_clean.str.rstrip('.')
data.name_clean = data.name_clean.str.rstrip('-')

data.name_clean = data.name_clean.apply(rep_sub_ch)

data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip(',')
data.name_clean = data.name_clean.str.rstrip('.')
data.name_clean = data.name_clean.str.rstrip('-')

In [382]:
data.name_clean = data.name_clean.apply(rep_digit)

data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip(',')
data.name_clean = data.name_clean.str.rstrip('.')
data.name_clean = data.name_clean.str.rstrip('-')

In [50]:
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.rstrip()
data.name_clean = data.name_clean.str.lstrip()
data.name_clean = data.name_clean.str.lstrip()
data.name_clean = data.name_clean.str.rstrip(',')
data.name_clean = data.name_clean.str.rstrip('-')

In [408]:
data[data.cid.str.contains('C11759')].name.values

array(['University College of Pharmacy, Thalappady, Kottayam'], dtype=object)

In [785]:
data[data.name_clean.str.contains('[0-9]{3}$')].name_clean.values
# data.name_clean = data.name_clean.str.replace(',$','.')
# data.name_clean.

array([], dtype=object)

In [644]:
# data[['name','name_clean']].to_csv('comp_name.csv')

In [589]:
data[data.name_clean.str.contains('P\.O\.')].name_clean.values

array([], dtype=object)

In [503]:
data[np.logical_xor(data.name.str.contains(r'\('),data.name.str.contains(r'\)'))].shape

(16, 51)

In [504]:
data[data.name.str.contains(r'[0-9]')].shape

(5153, 51)

In [505]:
data[data.name.str.contains('Ph. No')].shape

(70, 51)

In [506]:
# data.loc[data.state.isnull(),'state']='Tamil Nadu'
# data.state.unique()
# len(data.name.unique())
data[data.duplicated('name')].sort_values(by='name')
# data[data.name=='Vivekananda College of Education']
# len(data.name.unique())
# len(data.name)

,cid,name,address_line1,address_line2,city,Affiliated_to_University,state,pin_code,area,website,...,solar_power_generation,connectivity_nkn,connectivity_nmeict,no_of_books,no_of_journals,campus_friendly,grievance_redressal_mechanism,vigilance_cell,opportunity_cell,name_clean
11192,C24905,ABHYUDAYA DEGREE COLLEGE,"Govinda Nagar Colony,Behind Indira Nagar Colony",Chaparam,Srikakulam,"Dr.B.R.Ambedkar University, Etcherla",Andhra Pradesh,532001,2.070000,www.abhyudayadegreecollegesklm.com,...,False,False,False,1200.0,0.0,False,False,False,False,ABHYUDAYA DEGREE COLLEGE
11561,C25375,ADITYA DEGREE COLLEGE,Vijayawada,Near Chaitanya Techno School,Vijayawada,"Krishna University, Machhlipattanam",Andhra Pradesh,520001,2.000000,NaN,...,False,False,False,2100.0,2.0,False,False,False,False,ADITYA DEGREE COLLEGE
47157,U850,AISECT UNIVERSITY,MATWARI CHOWK,IN FRONT OF GANDHI MAIDAN,HAZARIBAG,NaN,Jharkhand,825301,0.000000,www.aisectuniversityjharkhand.ac.in,...,False,False,False,NaN,NaN,False,False,False,False,AISECT UNIVERSITY
27740,C48839,AKSHARA DEGREE COLLEGE,MIG-64 APHB COLONY MEDCHAL(MDL)R.R.DIST.,MIG-64 APHB COLONY MEDCHAL(MDL)R.R.DIST.,MEDCHAL,"Osmania University, Hyderabad",Telangana,501401,0.000000,NaN,...,False,False,False,500.0,NaN,False,False,False,False,AKSHARA DEGREE COLLEGE
25899,C46346,"ALIGARH COLLEGE OF ENGINEERING AND TECHNOLOGY,...","2.5 Km From Sasni Gate, Mathura Road",Aligarh,Aligarh,"Uttar Pradesh Technical University, Lucknow",Uttar Pradesh,202001,10.000000,www.acetup.org,...,False,False,False,42827.0,88.0,True,True,True,True,"ALIGARH COLLEGE OF ENGINEERING AND TECHNOLOGY,..."
40153,ST14545,ASIAN BUSINESS SCHOOL,Plot A2,Sector 125,Noida,NaN,Uttar Pradesh,201303,2.000000,http://www.abs.edu.in,...,False,False,False,3761.0,38.0,True,True,True,True,ASIAN BUSINESS SCHOOL
10467,C23117,Academy of Management Studies,"side of Nh-18, Gangraj,",P.S.Kuliana,Baripada,"Utkal University of Culture, Bhubaneswar",Odisha,757105,0.960000,www.amstudies.co.in,...,False,False,False,1229.0,25.0,True,False,False,True,Academy of Management Studies
12026,C25974,Academy of Management Studies,"S.No 192, Yenkapalli, Moinabad, Rangareddi Dis...","Behind Excise Academy, Himayatsagar Road, Near...",Hyderabad,"Osmania University, Hyderabad",Telangana,500086,2.500000,www.useams.org,...,False,False,True,2257.0,15.0,True,True,False,False,Academy of Management Studies
34550,C57212,Aditya College of Education,yerragondapalem,NaN,yerragondapalem,"Acharya Nagarjuna University, Guntur",Andhra Pradesh,523327,0.000000,NaN,...,False,False,False,NaN,NaN,False,False,False,False,Aditya College of Education
35898,C6978,Akshaya College of Education,"Ashokanagar,","2nd Cross,",Tumkur-02,"Tumkur University, Tumkur",Karnataka,572102,2.000000,www.acte-tumkur.com,...,False,False,False,5000.0,18.0,False,False,False,False,Akshaya College of Education


,name
0,Manish Institute of Computer Studies
1,"SIDDARTHA COLLEGE OF EDUCATION, JAMKHANDI"
2,"VVS'S ARTS/COMM COLLEGE, BIJAPUR"
3,"GOVT. FIRST GRADE COLLEGE, KOKATANOOR"
4,"S S S S'S COLLEGE OF BUSINESS ADMINISTRATION, ..."


In [170]:
names = pd.DataFrame(clgs['name'].apply(syn_gen))

In [171]:
names.head()

,name
0,"[MICS, manish institute of computer studies, m..."
1,"[SCE/J, siddartha college education, SCE jamk..."
2,"[VAC, VSACCB, VSACC bijapur, VAC-B, VAC B, VS..."
3,"[GFGC-K, GFGC/K, GFGC K, GFGC kokatanoor, gov..."
4,[s s s s's college business administration be...


In [173]:
len(names.iloc[4]['name'])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [174]:
names.name.values.tolist()
names[list(range(25))] = pd.DataFrame(names.name.values.tolist(), index= names.index)

In [178]:
names['name']=data['name']

In [179]:
def comma(s):
    if str(s) != str(None):
        return '"'+s+'"'
    return None

In [180]:
names = names.apply(comma)

In [181]:
names[names[24].notnull()]

,name,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24


In [182]:
names = names.drop([24],axis=1)

In [183]:
names.to_csv('clg_entities.csv',header=None,index=None)

rm_brk('kunal')

In [157]:
rm_brk('kuna(kd)l')

'kunal'

In [166]:
data['name'] = data['name'].apply(rm_brk)

In [185]:
data.to_csv('all_clgs.csv')

In [186]:
names.head()

,name,0,1,2,3,4,5,6,7,8,...,14,15,16,17,18,19,20,21,22,23
0,"""Manish Institute of Computer Studies""","""MICS""","""manish institute of computer studies""","""manish institute computer studies""",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"""SIDDARTHA COLLEGE OF EDUCATION, JAMKHANDI""","""SCE/J""","""siddartha college education""","""SCE jamkhandi""","""SCE-J""","""siddartha college of education""","""siddartha college education jamkhandi""","""siddartha college of education jamkhandi""","""SCE""","""SCE J""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"""VVS'S ARTS/COMM COLLEGE, BIJAPUR""","""VAC""","""VSACCB""","""VSACC bijapur""","""VAC-B""","""VAC B""","""VSACC/B""","""VSACC B""","""vvs's arts/comm college bijapur""","""VAC bijapur""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"""GOVT. FIRST GRADE COLLEGE, KOKATANOOR""","""GFGC-K""","""GFGC/K""","""GFGC K""","""GFGC kokatanoor""","""govt. first grade college kokatanoor""","""govt. first grade college""","""GFGC""","""GFGCK""",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"""S S S S'S COLLEGE OF BUSINESS ADMINISTRATION,...","""s s s s's college business administration be...","""SSSSCBA B""","""s s s s's college of business administration ...","""SSSSCBA/B""","""SSSSSCBA/B""","""SSSSCBA belgaum""","""SSSSCBA-B""","""SSSSSCBA""","""s s s s's college of business administration""",...,"""s s s s's college business administration""","""SSSSSCBA-B""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


college_entity.ipynb               merger.ipynb
college_front_page_scrapper.ipynb  scrapper2.ipynb
entity_generator.ipynb             temp_mapper.ipynb
entity_synonym_generator.ipynb     univ_names
mapper.ipynb


In [5]:
data = pd.read_csv('CollegeName_entity.csv')

In [10]:
data.iloc[:15000].to_csv('CollgeName1',header=None,index=None)

In [11]:
data.iloc[15000:].to_csv('CollegeName2',header=None,index=None)